### Import Libraries

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import iqr # For Finding Volume Outliers
from nsepython import *

import plotly.graph_objects as go
from plotly.subplots import make_subplots

Read the CSV file

In [2]:
## Stock Symbols contains list of 1250 NSE stock symbols (Need to update it every quarter / year)

stock_data = pd.read_csv('stock_data.csv')
stock_data.set_index('SYMBOL', inplace=True)
len(stock_data)

1435

### Tracking Stocks

In [3]:
tracking_stocks = ['KOTAKBANK', 'BAJFINANCE', 'DIVISLAB', 'DEEPAKNTR','APTUS', 'AETHER', 'ORIENTELEC','BUTTERFLY','INDOAMIN',
                  'VARROC', 'YASHO', 'ALKYLAMINE', 'DRREDDY','SCHNEIDER', 'ROUTE', 'GLS', 'LUPIN', 'VEDL','GLAND',
                  'CAMPUS','REDTAPE','SYMPHONY','BOROLTD', 'IKIO','FINOPB','CONCORDBIO','AMIORG','SOLARA','DREAMFOLKS',
                  'ATL','AVALON', 'RISHABH', 'VINATIORGA','IGPL','DELHIVERY', 'ALLCARGO','TCI','TREL','DMART','KALAMANDIR',
                  'ATGL','PETRONET','IGL','EASEMYTRIP','SHARDACROP','NACLIND','HONAUT','TITAGARH','KAYNES','UNIPARTS',
                  'HONDAPOWER','PAGEIND', 'PAYTM','PRAJIND','ANUP','GMMPFAUDLR','CYIENTDLM','IDEAFORGE', 'AZAD',
                  'TARSONS','MUTHOOTMF','IRCTC']

In [4]:
tracking_stocks = stock_data[stock_data.index.isin(tracking_stocks)]
tracking_stocks = tracking_stocks.sort_values('INDUSTRY')
tracking_stocks

,MARKET_CAP,INDUSTRY,SECTOR,MACRO,FNO
SYMBOL,,,,,
IDEAFORGE,3582.0,Aerospace & Defense,Capital Goods,Industrials,0.0
VARROC,8434.0,Auto Components,Automobile and Auto Components,Consumer Discretionary,0.0
KOTAKBANK,378426.0,Banks,Financial Services,Financial Services,1.0
FINOPB,2164.0,Banks,Financial Services,Financial Services,0.0
INDOAMIN,890.0,Chemicals & Petrochemicals,Chemicals,Commodities,0.0
...,...,...,...,...,...
ATL,1286.0,Transport Infrastructure,Services,Services,0.0
DELHIVERY,28586.0,Transport Services,Services,Services,0.0
TREL,1115.0,Transport Services,Services,Services,0.0


### Tracking Stocks Plot

In [5]:
try:
    figure_html = open('tracking.html', 'w')

    for symbol in tracking_stocks.index.values:
        
        try:
            if(symbol not in stock_data.index):
                print(f"{symbol} is not part of our stock data!")
                continue
            data = nse_past_results(symbol)

            df = pd.DataFrame(data.get('resCmpData'))
            isBank = data.get('bankNonBnking') == 'B'
    
            if(not isBank):
                if len(df.columns) == 0:
                    print(f"{symbol} no past results found for this symbol")
                    continue

                new_columns = {
                    're_to_dt': 'DATE',
                    're_create_dt': 'RELEASE_DATE',
                    're_net_sale': 'INCOME',
                    're_oth_tot_exp': 'COST',
                    're_oth_inc_new': 'OTHER_INCOME',
                    're_pro_loss_bef_tax': 'PBT',
                    're_net_profit': 'PROFIT',
                    're_basic_eps_for_cont_dic_opr': 'EPS'
                }

                df.rename(columns=new_columns, inplace=True)

                df = df[['DATE', 'RELEASE_DATE', 'INCOME', 'COST', 'OTHER_INCOME', 'PBT', 'PROFIT', 'EPS']]
                df.set_index('DATE', inplace=True)

                for col in df.columns:
                    if col != 'RELEASE_DATE':
                        df[col] = pd.to_numeric(df[col])

                for col in df.columns:
                    if col not in ['EPS', 'RELEASE_DATE']:
                        df[col] = round(df[col] / 100)
                        df[col] = df[col].round(0).astype('Int64')

                df['OP_INCOME'] = df['INCOME'] - df['COST']
                df['OPM'] = (100 * (df['INCOME'] - df['COST']) / df['INCOME']).round(2).astype(str) + "%"
                df['TAX'] = (100 * (df['PBT'] - df['PROFIT']) / df['PBT']).round(2).astype(str) + "%"

                df = df[['INCOME', 'OP_INCOME', 'OPM', 'OTHER_INCOME', 'TAX', 'PROFIT', 'EPS', 'RELEASE_DATE']]

                df = df.astype(str)
                df = df.transpose()
                df = df.iloc[:, ::-1]
            else:
                new_columns = {'re_to_dt': 'DATE', 
                   're_create_dt': 'RELEASE_DATE', 
                   're_int_earned':'NET_INTEREST_INCOME',
                   're_tot_exp_exc_pro_cont':'OPERATING_EXPENSE', 
                   're_oth_inc':'OTHER_INCOME',
                   're_oth_pro_cont':'PROVISION',
                   're_pro_loss_bef_tax':'PBT', 
                   're_con_pro_loss':'PROFIT', 
                   're_basic_eps':'EPS'}
    
                df.rename(columns=new_columns, inplace=True)
    
                df = df[['DATE', 'RELEASE_DATE', 'NET_INTEREST_INCOME', 'OPERATING_EXPENSE', 'PROVISION', 
                         'OTHER_INCOME','PBT', 'PROFIT', 'EPS']]
                df.set_index('DATE', inplace=True)
    
                for col in df.columns:
                    if col != 'RELEASE_DATE':
                        df[col] = pd.to_numeric(df[col])
    
                for col in df.columns:
                    if col not in ['EPS','RELEASE_DATE']:
                        df[col] = round(df[col] / 100)
                        df[col] = df[col].round(0).astype('Int64')
            
                df['OP_INCOME'] = df['NET_INTEREST_INCOME'] - df['OPERATING_EXPENSE']
                df['OPM'] = (100 * (df['NET_INTEREST_INCOME'] - df['OPERATING_EXPENSE']) / df['NET_INTEREST_INCOME']).round(2).astype(str) + "%"
                df['TAX'] = (100 * (df['PBT'] - df['PROFIT']) / df['PBT']).round(2).astype(str) + "%"
    
                df = df[['NET_INTEREST_INCOME', 'OP_INCOME', 'OPM','OTHER_INCOME', 'PROVISION', 'TAX', 'PROFIT', 'EPS', 'RELEASE_DATE']]
    
                df = df.astype(str)
                df = df.transpose()
                df = df.iloc[:, ::-1]
        
    
            ### Ploting 
    
            data = yf.download(symbol + ".NS", start='2023-01-01', end=pd.to_datetime('today')+pd.DateOffset(1), progress=False)
    
            data['20DMA'] = data['Close'].rolling(window=20).mean()
            data['50DMA']= data['Close'].rolling(window=50).mean()
            data['100DMA']= data['Close'].rolling(window=100).mean()
    
            data['diff'] = data['Close'] - data['Open']
            data.loc[data['diff'] >= 0, 'color'] = 'green'
            data.loc[data['diff'] < 0, 'color'] = 'red'
    
            plot_data = data[data.index > (pd.to_datetime('today') - pd.DateOffset(days=270))]

            figure = make_subplots(specs=[[{"secondary_y": True}]])
    
            # Subplot 1 - Price Candlestick
    
            figure.add_trace(go.Candlestick(x = plot_data.index,
                              open = plot_data['Open'],
                              high = plot_data['High'],
                              low = plot_data['Low'],
                              close = plot_data['Close'],
                              name='Price'))
            figure.update_yaxes(range=[plot_data['Close'].min()*0.9, plot_data['Close'].max()*1.05])
            figure.update_xaxes(rangebreaks = [ dict(bounds=['sat','mon']) ] ) # hide weekends 
            figure.update_layout(title={'text':symbol, 'x':0.5})
            figure.update_layout(xaxis_rangeslider_visible=False)  #hide range slider
    
            # Subplot 2 - Volume and DMAs
    
            figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['20DMA'], marker_color='blue',name='20 Day MA'))
            figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['50DMA'], marker_color='orange',name='50 Day MA'))
            figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['100DMA'], marker_color='green',name='100 Day MA'))
    
            figure.add_trace(go.Bar(x=plot_data.index, y=plot_data['Volume'], name='Volume', marker={'color':plot_data['color']}),secondary_y=True)
            figure.update_yaxes(range=[0, plot_data['Volume'].max()*5], secondary_y=True)
            figure.update_yaxes(visible=False, secondary_y=True)
    
            # Adding Text Below
            stock_row = stock_data.loc[symbol]
            figure.add_annotation(dict(font=dict(color='black', size=16.5),
                x=0.5,  # Center aligned horizontally
                y=-0.12, showarrow=False,
                text="SECTOR - " + stock_row['SECTOR']+"  MARKET CAP - "+str(stock_row['MARKET_CAP'].astype(int)) + "Cr",
                textangle=0,
                xanchor='center',  # Center aligned horizontally
                yanchor='bottom',  # Aligned to the bottom
                xref="paper", yref="paper"))
    
            # Adding Earnings Date in the figure
            if('31-DEC-2023' in df.columns):
                figure.add_annotation(go.layout.Annotation(text="Q3",
                    x=pd.to_datetime(df.loc['RELEASE_DATE']['31-DEC-2023']),
                    y=plot_data['Close'].min()*0.9, 
                    showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=3, arrowcolor="yellow",
                    ax=0, ay=-40))
    
            figure.add_annotation(go.layout.Annotation(text="Q2",
                x=pd.to_datetime(df.loc['RELEASE_DATE']['30-SEP-2023']),
                y=plot_data['Close'].min()*0.9, 
                showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=3, arrowcolor="yellow",
                ax=0, ay=-40))
    
    
            figure.add_annotation(go.layout.Annotation(text="Q1",
                x=pd.to_datetime(df.loc['RELEASE_DATE']['30-JUN-2023']),
                y=plot_data['Close'].min()*0.9, 
                showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=3, arrowcolor="yellow",
                ax=0, ay=-40))
    
    
            html_table = df.style.set_table_styles([
            {'selector': 'thead th', 'props': [('background-color', 'lightblue'), ('color', 'black'), ('text-align', 'center'), ('font-weight', 'bold')]},
            {'selector': 'tbody td', 'props': [('text-align', 'center')]}]).render()
    
            ### Writing the results dataframe and plot to the html page
            figure_html.write(html_table)
            figure_html.write(figure.to_html(full_html=False))
        
        except Exception as symbol_exception:
            print(f"Error processing symbol {symbol}: {symbol_exception}")
            continue
except Exception as main_exception:
    print(f"Main error: {main_exception}")

finally:
    # Close the HTML file
    figure_html.close()

print('Execution Completed!!!!')

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipy

Error processing symbol YASHO: '30-JUN-2023'


/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



REDTAPE no past results found for this symbol


/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipy

...

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



TREL no past results found for this symbol


/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



Execution Completed!!!!


/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_9260/2590077612.py:165: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

